### About

**NOTE: This notebook is a continuation of my previous notebook which contained EDA.**   

You can refer the following links:  
- Part-1 EDA: LINK  
- End-to-End implementation with deployment on AWS: https://github.com/Sharma-Ayush/Credit-Card-Fraud-Detection.git

### Objective of this notebook

The objective for this notebook is:  
- Feature Engineering.  
- Handling Imbalance.  
- Feature importance.  
- Cross-validation and hyper parameter tuning of classification models.  
- Testing on test set.

<u>PS:</u>  
- Feel free to contact me if you have any doubts or feedback through the comment section or my socials.
- Please upvote the notebook if you like it, as it would motivate me to develop more projects like these.

### My Socials

Follow me on these platforms, for more such content:  

LinkedIn: https://www.linkedin.com/in/ayush-sharma-660831125/  
Github: https://github.com/Sharma-Ayush  
Kaggle: https://www.kaggle.com/ayushsharma0812

### Import Required Libraries

In [38]:
# Data Manipulation
import numpy as np
import pandas as pd

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import ADASYN, SMOTE

from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from imblearn.pipeline import Pipeline

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

from sklearn.metrics import auc, precision_recall_curve
from sklearn.metrics._scorer import make_scorer

# extra modules
from warnings import filterwarnings
filterwarnings("ignore")


### Custom classes & functions

In [2]:
def double_log_transform(x):
    '''Transform x by taking the log of the data after shifting by 1. This operation is done two times iteratively.'''
    return np.log10(np.log10(x + 1) + 1)

def cube_root_transform(x):
    '''Transform x by taking the cube root of the data.'''
    return np.cbrt(x)

In [3]:
def area_under_precision_recall_curve(y_true, y_pred):
    '''Function that can compute the area under precision recall curve'''
    precision, recall, thresholds = precision_recall_curve(y_true, y_pred)
    auc_precision_recall = auc(recall, precision)
    return auc_precision_recall


In [4]:
def find_best_cv_params(X, Y, training_pipeline, params, scoring, score_name, n_iter = 25, search_type = 'random', cv = 5):
    '''Perform random search or grid search on given X and Y using given model & parameter distributions preceeded by column_trasnformer, standard_scaler & resampler.
    n_iter(for random search), cv(no. of splits), scoring(score for finding best params) can be passed as parameters.'''
    
    if search_type == 'random':
        cross_validator = RandomizedSearchCV(training_pipeline,
                                             param_distributions = params,
                                             n_iter = n_iter,
                                             cv = cv,
                                             scoring = scoring,
                                             return_train_score = True,
                                             refit = False,
                                             error_score = 0,
                                             n_jobs = -1,
                                             random_state = 42)
    elif search_type == 'grid':
        cross_validator = GridSearchCV(training_pipeline,
                                       param_grid = params,
                                       scoring = scoring,
                                       cv = cv,
                                       n_jobs = -1,
                                       refit = False,
                                       error_score = 0,
                                       return_train_score = True)
    else:
        print('Invalid search type. Aborting.')
        return None

    cross_validator.fit(X, Y)
    print(f'For model - {training_pipeline.steps[-1][0]} {search_type} search:')
    print('----------------------------------------------------------')
    print(f'Best mean {score_name}:')
    print(f'Train set: {np.round(max(cross_validator.cv_results_['mean_train_score']), 3)}')
    print(f'Validation set: {np.round(cross_validator.best_score_, 3)}')
    print('----------------------------------------------------------')

    if search_type == 'random':
        return cross_validator.best_params_
    elif search_type == 'grid':
        print(cross_validator.best_params_)
        return cross_validator.best_params_

In [5]:
def plot_boxplot_for_multi_num_cols(data, num_cols, target_col, color_mapping = None):
    '''Plot boxplot of numerical columns segregated per category of target_col with given color mapping for multiple numerical columns.'''
    
    # Calculate number of figures needed for 5 columns per figure or lesser
    no_of_figures = len(num_cols) // 5

    if len(num_cols) % 5 != 0:
        no_of_figures += 1

    # Plotting the histograms and density plots
    for i in range(no_of_figures):
        columns_to_plot = num_cols[i*5:(i+1)*5]

        fig, ax = plt.subplots(1, len(columns_to_plot), figsize = (len(columns_to_plot)*6, 4), sharex = True)

        for index, col in enumerate(columns_to_plot):

            if color_mapping != None:
                sns.boxplot(data, y = col, hue = target_col, ax = ax[index], palette = color_mapping)
            else:
                sns.boxplot(data, y = col, hue = target_col, ax = ax[index])
            ax[index].set_title(col)

        plt.subplots_adjust(wspace = 0.25)

        plt.show();

### Loading the dataset

In [6]:
df_train = pd.read_csv('Data/creditcard_train.csv')

### Handling imbalance

We have already seen the huge imbalance that exists within our dataset. We need to mitigate that imbalance, otherwise our model will focus more on the majority class(genuine transactions). There are many techniques that we can use:  

1. <u>Undersampling of majority class:</u> We take random sample of majority class so that the size matches that of minority class. This will reduce the dataset size which is good if we want reduced computation effort and thus, will lead to faster computation but at the same time it will lead to loss of information.  

2. <u>Oversampling of minority class:</u> We do random sampling on minority class with replacement so that the size of minority class matches that of majority class. This will increase dataset's size which will lead to more computational effort but will preserve information unlike undersampling. Duplicates are generated within minority class and this can lead to overfitting.  

3. <u>Synthetic data generation techniques:</u> We use techniques like SMOTE, its variants or ADASYN to oversample minority class in a systematic way by adding points in between existing minority class data points with some logic behind. This is the best way to upsample as duplication is not present but points are added along straight lines in between existing data points and this can lead to addition of artificial patterns. 

4. <u>Weight based sensitivity:</u> Many algorithms allow us to specify weights for each record of our dataset, we can specify higher weights to minority class records and the algorithm will focus more on them.

I would like to try both synthetic generation and weight based sensitivity, then compare them but the dataset is decently big in size and I want to perform cross-validaiton for hyper parameter tuning as well and thus, there is a problem here.  

When we generate cross-validation sets they should be first split up and then each of the set should be passed through the synthetic generator, any other transformations and then finally evaluated upon. This is done after splitting so that the information doesn't leak in between train and validation sets. To save time on computation and still do hyperparameter tuning which itself will be time taking, I want a technique of synthetic generation where the size of the resulting dataset is not too big because I don't have that much time and computation to spare on both - mulltiple synthetic generations and cross-validation.  

So, what should I do? Definitely, I can try weight based sensitivity as it is, because it does not involve any synthetic generation. Other option that I am thinking of is a combination of undersampling and ADASYN. If I was to undersample majority class all the way to same size as minority class then that will be too much loss of information. Due to my own time and computation constraints, I want to just reduce the size of majority class by enough and at the same time oversample minority class through synthetic generator such that the time taken for this process is managable.

I will use ADASYN. ADSYN tries to add more data towards the low density points of minority class. This means that even noisy points of fraudulent class will get attention and it might overfit a bit too much on the minority class i.e. it might perform very well on minority class but poorer on majority class. In a case like this of frauds and scams we want to primarily focus on predicting fraud cases as best as possible and even if some genuine cases(as long as its not too much above some threshold) are predicted to be as fraud we should be okay with it as the credi card issuer can provide tighter verification to user in these cases. This way we have tight leash on frauds which will save the company money and tighter verification for genuine cases predicted as fraud. If its too much overfitting we can try other techniques like SMOTE.

Undersampling will be done before training. I will choose a decent size of 10k for undersampling and then upsample minority class to this using ADASYN. Anyone who can spare time and computational effort on this can use the whole dataset as it is and upsample minority class, this will preserve information.

In [7]:
minority_class_size = df_train.Class.value_counts().min()

# Random under sampling the majority class to a size of 10000
X_resampled, Y_resampled = RandomUnderSampler(sampling_strategy = minority_class_size/10000, random_state = 42).fit_resample(df_train.drop(columns = 'Class'), df_train['Class'])

# ADASYN resampler fo synthetic generation of minority class
ADASYN_resampler = ADASYN(sampling_strategy = 'auto', random_state = 42)

### Feature Engineering

Based on the EDA notebook, I have to perform following transformations and manipulations on my dataset:  
- Take log10 Amount column after shiting the data by 1. This is done two times iteratively.  
- Drop 3 columns: V13, V15, and V23.  
- Take cube root of all the rest of PCA encoded columns.  

The order of the transformations will be as followed:  
- Transform columns based on EDA.  
- Standardize columns so that scales of columns are similar.  
- Then in the last use ADASYN for upsampling minority class.

In [8]:
# Define the columns to be transformed
double_log_transform_columns = ['Amount']
cube_root_transform_columns = ['V' + str(i) for i in range(1, 29) if i not in [13, 15, 23]]
passthrough_columns = ['Time']
drop_columns = ['V13', 'V15', 'V23']

# Define the functional transformers for the group of columns
double_log_transformer = FunctionTransformer(func = double_log_transform, feature_names_out = 'one-to-one')
cube_root_transformer = FunctionTransformer(func = cube_root_transform, feature_names_out = 'one-to-one')
pass_through_transformer = 'passthrough'

# Create transformers
column_transformer = ColumnTransformer([('double_log_transform', double_log_transformer, double_log_transform_columns),
                                        ('cube_root_transformer', cube_root_transformer, cube_root_transform_columns),
                                        ('passthrough', pass_through_transformer, passthrough_columns)],
                                        remainder = 'drop')
standard_scaler = StandardScaler()

### Predictive Modeling

**What metric to choose for evaluation?**  
Since its an imbalanced dataset, we can't use accuracy as an evaluation metric as that would be highly influenced by the majority class.  

If we were to work on this problem statement in a business setting, then we would have some criteria on precision and recall that the final model should achieve to be actually useful and deployed into production. Since we want to focus more on fraud to keep chargeback costs to lower amount, we would have had stricter criteria on recall so that the prediction within fraud class is quite good and a more relaxed criteria on precision meaning that we can allow more errors in genuine cases. But these are two metrics, how can we use two metrics to properly compare several models? That's why measures which combine these precision and recall into one measure are used for model comparison. Once the final model in decided based on this summarization measure, we can adjust the threshold on the soft probability to adjust precision and recall levels of the model. If then we are able to achieve precision and recall above our business criterias, we can go forward to deploying our model otherwise we go back to the drawing board.  

For evaluating, imbalanced datasets F1 score, area under the ROC and area under the precision-recall curve are common measures. For very high imbalanced dataset, precision-recall curve area is seen to be more sensitive and thus, better towards summarizing the performance. Therefore, F1 score or area under the precision-recall curve can be used here. As per the datasets author they have suggested to use area under precision-recall curve and that's what I will use here.

In [9]:
# Defining the evaluation metric scorer object
area_under_precision_recall_curve_scorer = make_scorer(area_under_precision_recall_curve, response_method = 'predict_proba', greater_is_better = True)

#### Experimenting

For experimental purpose, I will just use logistic regression model.  

I will try to see the following things:  
1. Class weight balancing vs ADASYN resampler  
2. Does column transformation have any effect on performance or not?

##### 1. Class weight balancing

In [10]:
# Performing random search cross-validation

find_best_cv_params(X_resampled,
                    Y_resampled,
                    Pipeline([('column_transformer', column_transformer), 
                              ('standard_scaler', standard_scaler),
                              ('Logistic_Regression', LogisticRegression(class_weight = 'balanced', n_jobs = -1, random_state = 123))]), 
                    {'Logistic_Regression__penalty' : ['l1', 'l2', 'elasticnet', None],
                     'Logistic_Regression__C' : np.logspace(-4, 4, 20), 
                     'Logistic_Regression__solver': ['lbfgs', 'newton-cg', 'sag', 'saga'],
                     'Logistic_Regression__l1_ratio': np.linspace(0.1, 1, 9, endpoint = False)}, 
                    area_under_precision_recall_curve_scorer,
                    'area under precision recall curve',
                    n_iter = 50)

For model - Logistic_Regression random search:
----------------------------------------------------------
Best mean area under precision recall curve:
Train set: 0.927
Validation set: 0.914
----------------------------------------------------------


c:\Users\adity\anaconda3\envs\venv\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
110 fits failed out of a total of 250.
The score on these train-test partitions for these parameters will be set to 0.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\adity\anaconda3\envs\venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\adity\anaconda3\envs\venv\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\adity\anaconda3\envs\venv\Lib\site-packages\imblearn\pipeline.py", line 

{'Logistic_Regression__solver': 'sag',
 'Logistic_Regression__penalty': 'l2',
 'Logistic_Regression__l1_ratio': np.float64(0.8),
 'Logistic_Regression__C': np.float64(0.012742749857031334)}

In [11]:
# Performing grid search cross-validation

best_params = find_best_cv_params(X_resampled,
                                  Y_resampled,
                    Pipeline([('column_transformer', column_transformer), 
                              ('standard_scaler', standard_scaler),
                              ('Logistic_Regression', LogisticRegression(class_weight = 'balanced', n_jobs = -1, random_state = 123))]),
                                  {'Logistic_Regression__penalty' : ['elasticnet'],
                                  'Logistic_Regression__C' : np.logspace(-3, -1, 10), 
                                  'Logistic_Regression__solver': ['saga'],
                                  'Logistic_Regression__l1_ratio': np.linspace(0.5, 0.95, 10, endpoint = True)}, 
                                  area_under_precision_recall_curve_scorer,
                                  'area under precision recall curve',
                                  search_type = 'grid')

For model - Logistic_Regression grid search:
----------------------------------------------------------
Best mean area under precision recall curve:
Train set: 0.926
Validation set: 0.916
----------------------------------------------------------
{'Logistic_Regression__C': np.float64(0.021544346900318832), 'Logistic_Regression__l1_ratio': np.float64(0.55), 'Logistic_Regression__penalty': 'elasticnet', 'Logistic_Regression__solver': 'saga'}


##### 2. ADASYN resampler

In [12]:
# Performing random search cross-validation

find_best_cv_params(X_resampled,
                    Y_resampled,
                    Pipeline([('column_transformer', column_transformer), 
                              ('standard_scaler', standard_scaler),
                              ('ADASYN_resample', ADASYN_resampler),
                              ('Logistic_Regression', LogisticRegression(n_jobs = -1, random_state = 123))]),
                    {'Logistic_Regression__penalty' : ['l1', 'l2', 'elasticnet', None],
                     'Logistic_Regression__C' : np.logspace(-4, 4, 20), 
                     'Logistic_Regression__solver': ['lbfgs', 'newton-cg', 'sag', 'saga'],
                     'Logistic_Regression__l1_ratio': np.linspace(0.1, 1, 9, endpoint = False)}, 
                    area_under_precision_recall_curve_scorer,
                    'area under precision recall curve',
                    n_iter = 50)

For model - Logistic_Regression random search:
----------------------------------------------------------
Best mean area under precision recall curve:
Train set: 0.753
Validation set: 0.728
----------------------------------------------------------


c:\Users\adity\anaconda3\envs\venv\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
110 fits failed out of a total of 250.
The score on these train-test partitions for these parameters will be set to 0.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\adity\anaconda3\envs\venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\adity\anaconda3\envs\venv\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\adity\anaconda3\envs\venv\Lib\site-packages\imblearn\pipeline.py", line 

{'Logistic_Regression__solver': 'newton-cg',
 'Logistic_Regression__penalty': None,
 'Logistic_Regression__l1_ratio': np.float64(0.1),
 'Logistic_Regression__C': np.float64(1.623776739188721)}

Seems like its undefitting, even though the logistic regression model has no regularization i.e. the complexity of model is at its maximum it can be. Does it mean that using ADASYN has introduced artificial noise thus making the patterns complexer? Because if data has become more complex and model is simpler that would lead to underfitting. Maybe we can also play with parameters of ADASYN and hope things change but I won't try this here. Rather I want to try SMOTE.

##### 3. SMOTE resampler

In [13]:
SMOTE_resampler = SMOTE(random_state = 42)

# Performing random search cross-validation

find_best_cv_params(X_resampled,
                    Y_resampled,
                    Pipeline([('column_transformer', column_transformer), 
                              ('standard_scaler', standard_scaler),
                              ('SMOTE_resample', SMOTE_resampler),
                              ('Logistic_Regression', LogisticRegression(n_jobs = -1, random_state = 123))]),
                    {'Logistic_Regression__penalty' : ['l1', 'l2', 'elasticnet', None],
                     'Logistic_Regression__C' : np.logspace(-4, 4, 20), 
                     'Logistic_Regression__solver': ['lbfgs', 'newton-cg', 'sag', 'saga'],
                     'Logistic_Regression__l1_ratio': np.linspace(0.1, 1, 9, endpoint = False)}, 
                    area_under_precision_recall_curve_scorer,
                    'area under precision recall curve',
                    n_iter = 50)

For model - Logistic_Regression random search:
----------------------------------------------------------
Best mean area under precision recall curve:
Train set: 0.925
Validation set: 0.912
----------------------------------------------------------


c:\Users\adity\anaconda3\envs\venv\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
110 fits failed out of a total of 250.
The score on these train-test partitions for these parameters will be set to 0.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\adity\anaconda3\envs\venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\adity\anaconda3\envs\venv\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\adity\anaconda3\envs\venv\Lib\site-packages\imblearn\pipeline.py", line 

{'Logistic_Regression__solver': 'sag',
 'Logistic_Regression__penalty': 'l2',
 'Logistic_Regression__l1_ratio': np.float64(0.8),
 'Logistic_Regression__C': np.float64(0.012742749857031334)}

SMOTE tends to work much better than ADASYN in this problem statement. ADASYN tries to add artificial points more and more near  the exisitng points which are farther from the center of minority class i.e. points which are mostly surrounded by majority class points. But what if these points are pure noisy points which we don't want to focus on then ADASYN will force us to focus on those points unnecessarily and this will affect our performance. SMOTE on the other hand adds new points randomly.

In [14]:
# Performing grid search cross-validation

best_params = find_best_cv_params(X_resampled,
                                  Y_resampled,
                                  Pipeline([('column_transformer', column_transformer), 
                                              ('standard_scaler', standard_scaler),
                                              ('SMOTE_resample', SMOTE_resampler),
                                              ('Logistic_Regression', LogisticRegression(n_jobs = -1, random_state = 123))]),
                                  {'Logistic_Regression__penalty' : ['elasticnet'],
                                  'Logistic_Regression__C' : np.logspace(-3, -1, 10), 
                                  'Logistic_Regression__solver': ['saga'],
                                  'Logistic_Regression__l1_ratio': np.linspace(0.5, 0.95, 10, endpoint = True)}, 
                                  area_under_precision_recall_curve_scorer,
                                  'area under precision recall curve',
                                  search_type = 'grid')

For model - Logistic_Regression grid search:
----------------------------------------------------------
Best mean area under precision recall curve:
Train set: 0.925
Validation set: 0.915
----------------------------------------------------------
{'Logistic_Regression__C': np.float64(0.007742636826811269), 'Logistic_Regression__l1_ratio': np.float64(0.6), 'Logistic_Regression__penalty': 'elasticnet', 'Logistic_Regression__solver': 'saga'}


Class weight balancing performs slighlty better than SMOTE. Going on ahead we will work with class weight balancing only, this will reduce the need for resampling in our training pipeline. I can also increase my resampling size for the majority class and that will help in adding more information but I will stick with this for now.

##### 4. Class weight balancing without Amount column transformation

In [15]:
column_transformer_1 = ColumnTransformer([('cube_root_transform', cube_root_transformer, cube_root_transform_columns),
                                          ('drop', 'drop', drop_columns)],
                                          remainder = 'passthrough')

# Performing random search cross-validation

find_best_cv_params(X_resampled,
                    Y_resampled,
                    Pipeline([('column_transformer', column_transformer_1), 
                              ('standard_scaler', standard_scaler),
                              ('Logistic_Regression', LogisticRegression(class_weight = 'balanced', n_jobs = -1, random_state = 123))]),
                    {'Logistic_Regression__penalty' : ['l1', 'l2', 'elasticnet', None],
                     'Logistic_Regression__C' : np.logspace(-4, 4, 20), 
                     'Logistic_Regression__solver': ['lbfgs', 'newton-cg', 'sag', 'saga'],
                     'Logistic_Regression__l1_ratio': np.linspace(0.1, 1, 9, endpoint = False)}, 
                    area_under_precision_recall_curve_scorer,
                    'area under precision recall curve',
                    n_iter = 50)

For model - Logistic_Regression random search:
----------------------------------------------------------
Best mean area under precision recall curve:
Train set: 0.923
Validation set: 0.914
----------------------------------------------------------


c:\Users\adity\anaconda3\envs\venv\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
110 fits failed out of a total of 250.
The score on these train-test partitions for these parameters will be set to 0.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\adity\anaconda3\envs\venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\adity\anaconda3\envs\venv\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\adity\anaconda3\envs\venv\Lib\site-packages\imblearn\pipeline.py", line 

{'Logistic_Regression__solver': 'sag',
 'Logistic_Regression__penalty': 'l2',
 'Logistic_Regression__l1_ratio': np.float64(0.6),
 'Logistic_Regression__C': np.float64(0.08858667904100823)}

In [16]:
# Performing grid search cross-validation

best_params = find_best_cv_params(X_resampled,
                                  Y_resampled,
                                  Pipeline([('column_transformer', column_transformer_1), 
                                              ('standard_scaler', standard_scaler),
                                              ('Logistic_Regression', LogisticRegression(class_weight = 'balanced', n_jobs = -1, random_state = 123))]),
                                  {'Logistic_Regression__penalty' : ['l2'],
                                  'Logistic_Regression__C' : np.logspace(-2, 0, 20), 
                                  'Logistic_Regression__solver': ['sag']}, 
                                  area_under_precision_recall_curve_scorer,
                                  'area under precision recall curve',
                                  search_type = 'grid')

For model - Logistic_Regression grid search:
----------------------------------------------------------
Best mean area under precision recall curve:
Train set: 0.923
Validation set: 0.914
----------------------------------------------------------
{'Logistic_Regression__C': np.float64(0.026366508987303583), 'Logistic_Regression__penalty': 'l2', 'Logistic_Regression__solver': 'sag'}


There is only 0.2% decrease in AUC score for precision-recall curve.

##### 5. Class weight balancing without V1 to V28 column transformation

In [17]:
column_transformer_2 = ColumnTransformer([('double_log_transform', double_log_transformer, double_log_transform_columns),
                                          ('drop', 'drop', drop_columns)],
                                          remainder = 'passthrough')

# Performing random search cross-validation

find_best_cv_params(X_resampled,
                    Y_resampled,
                    Pipeline([('column_transformer', column_transformer_2), 
                              ('standard_scaler', standard_scaler),
                              ('Logistic_Regression', LogisticRegression(class_weight = 'balanced', n_jobs = -1, random_state = 123))]),
                    {'Logistic_Regression__penalty' : ['l1', 'l2', 'elasticnet', None],
                     'Logistic_Regression__C' : np.logspace(-4, 4, 20), 
                     'Logistic_Regression__solver': ['lbfgs', 'newton-cg', 'sag', 'saga'],
                     'Logistic_Regression__l1_ratio': np.linspace(0.1, 1, 9, endpoint = False)}, 
                    area_under_precision_recall_curve_scorer,
                    'area under precision recall curve',
                    n_iter = 50)

For model - Logistic_Regression random search:
----------------------------------------------------------
Best mean area under precision recall curve:
Train set: 0.936
Validation set: 0.924
----------------------------------------------------------


c:\Users\adity\anaconda3\envs\venv\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
110 fits failed out of a total of 250.
The score on these train-test partitions for these parameters will be set to 0.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\adity\anaconda3\envs\venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\adity\anaconda3\envs\venv\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\adity\anaconda3\envs\venv\Lib\site-packages\imblearn\pipeline.py", line 

{'Logistic_Regression__solver': 'lbfgs',
 'Logistic_Regression__penalty': None,
 'Logistic_Regression__l1_ratio': np.float64(0.6),
 'Logistic_Regression__C': np.float64(206.913808111479)}

In [18]:
# Performing grid search cross-validation

best_params = find_best_cv_params(X_resampled,
                                  Y_resampled,
                                  Pipeline([('column_transformer', column_transformer_2), 
                                              ('standard_scaler', standard_scaler),
                                              ('Logistic_Regression', LogisticRegression(class_weight = 'balanced', n_jobs = -1, random_state = 123))]),
                                  {'Logistic_Regression__penalty' : [None],
                                  'Logistic_Regression__solver': ['lbfgs']}, 
                                  area_under_precision_recall_curve_scorer,
                                  'area under precision recall curve',
                                  search_type = 'grid')

For model - Logistic_Regression grid search:
----------------------------------------------------------
Best mean area under precision recall curve:
Train set: 0.936
Validation set: 0.924
----------------------------------------------------------
{'Logistic_Regression__penalty': None, 'Logistic_Regression__solver': 'lbfgs'}


The model performance has improved compared to when the transformations were performed. The transformation was done from the persepective that even extreme points should have similar weightage when it comes to training so that the algorithm can perform better throughout the range of these predictor features but this increase in performance implies that these extreme points on which when we focussed more lead to poorer performance overall, this means that these points could actually be outliers which don't follow similar pattern as majority of data points. Also, our model is quite simple and linear in nature. Thus, if the patterns become non-linear near these rextreme points then our model would not be able to model it appropriately, this means that we might need more complex models. Thus, I should try more complex model like decision tree first before we jump to more conclusions.

##### 6. Decision tree with class weight balancing

In [19]:
# Performing random search cross-validation

find_best_cv_params(X_resampled,
                    Y_resampled,
                    Pipeline([('column_transformer', column_transformer), 
                              ('standard_scaler', standard_scaler),
                              ('Decision_Tree', DecisionTreeClassifier(class_weight = 'balanced', random_state = 123))]),                    
                    {'Decision_Tree__max_features': [None, 'sqrt', 'log2'],
                     'Decision_Tree__min_samples_leaf': list(range(5, 51, 5)),
                     'Decision_Tree__max_depth' : list(range(1, 3, 1)),
                     'Decision_Tree__splitter': ['best', 'random'],
                     'Decision_Tree__criterion' :['gini', 'entropy', 'log_loss']}, 
                    area_under_precision_recall_curve_scorer,
                    'area under precision recall curve',
                    n_iter = 100)

For model - Decision_Tree random search:
----------------------------------------------------------
Best mean area under precision recall curve:
Train set: 0.891
Validation set: 0.879
----------------------------------------------------------


{'Decision_Tree__splitter': 'best',
 'Decision_Tree__min_samples_leaf': 10,
 'Decision_Tree__max_features': None,
 'Decision_Tree__max_depth': 2,
 'Decision_Tree__criterion': 'log_loss'}

In [20]:
# Performing random search cross-validation

best_params = find_best_cv_params(X_resampled,
                                  Y_resampled,
                                  Pipeline([('column_transformer', column_transformer), 
                                              ('standard_scaler', standard_scaler),
                                              ('Decision_Tree', DecisionTreeClassifier(class_weight = 'balanced', random_state = 123))]),                    
                                  {'Decision_Tree__max_features': [None, 'sqrt', 'log2'],
                                   'Decision_Tree__min_samples_leaf': list(range(1, 21, 2)),
                                   'Decision_Tree__max_depth' : list(range(2, 3, 1)),
                                   'Decision_Tree__splitter': ['best', 'random'],
                                   'Decision_Tree__criterion' :['gini', 'entropy', 'log_loss']}, 
                                   area_under_precision_recall_curve_scorer,
                                  'area under precision recall curve',
                                   search_type = 'grid')

For model - Decision_Tree grid search:
----------------------------------------------------------
Best mean area under precision recall curve:
Train set: 0.891
Validation set: 0.879
----------------------------------------------------------
{'Decision_Tree__criterion': 'entropy', 'Decision_Tree__max_depth': 2, 'Decision_Tree__max_features': None, 'Decision_Tree__min_samples_leaf': 1, 'Decision_Tree__splitter': 'best'}


Clearly, there is not too much complexity in the dataset decision boundaries based on following two observations:  
1. Logistic regression performed better than decision tree model.  
2. The depth of the tree is not too much.

But there is one problem with decision trees and that is it does piecewise approximation of the decision boundaries which can be detrimental for performance in case the decision boundaries are not parallel to the predictor axes and thus, it might even perfom poorer than logistic regression in this scenario. One thing that can be tried here is PCA which maybe help in the case that the decision boundaries are linear but not for when non-linearity comes in non-linear situations. Lets try PCA as well. But most of the columns went through PCA already, only two columns Amount and Time are new. Thus, it might not have much effect.

##### 7. Decision tree with class weight balancing and PCA

In [21]:
# Performing random search cross-validation

find_best_cv_params(X_resampled,
                    Y_resampled,
                    Pipeline([('column_transformer', column_transformer), 
                              ('pca', PCA(random_state = 42)),
                              ('standard_scaler', standard_scaler),
                              ('Decision_Tree', DecisionTreeClassifier(class_weight = 'balanced', random_state = 123))]),                    
                    {'Decision_Tree__max_features': [None, 'sqrt', 'log2'],
                     'Decision_Tree__min_samples_leaf': list(range(5, 51, 5)),
                     'Decision_Tree__max_depth' : list(range(1, 3, 1)),
                     'Decision_Tree__splitter': ['best', 'random'],
                     'Decision_Tree__criterion' :['gini', 'entropy', 'log_loss']}, 
                     area_under_precision_recall_curve_scorer,
                     'area under precision recall curve',
                     n_iter = 100)

For model - Decision_Tree random search:
----------------------------------------------------------
Best mean area under precision recall curve:
Train set: 0.893
Validation set: 0.886
----------------------------------------------------------


{'Decision_Tree__splitter': 'best',
 'Decision_Tree__min_samples_leaf': 10,
 'Decision_Tree__max_features': None,
 'Decision_Tree__max_depth': 2,
 'Decision_Tree__criterion': 'log_loss'}

In [22]:
# Performing random search cross-validation

best_params = find_best_cv_params(X_resampled,
                                  Y_resampled,
                                  Pipeline([('column_transformer', column_transformer), 
                                            ('pca', PCA(random_state = 42)),
                                            ('standard_scaler', standard_scaler),
                                            ('Decision_Tree', DecisionTreeClassifier(class_weight = 'balanced', random_state = 123))]),                    
                                  {'Decision_Tree__max_features': [None, 'sqrt', 'log2'],
                                   'Decision_Tree__min_samples_leaf': list(range(1, 21, 2)),
                                   'Decision_Tree__max_depth' : list(range(2, 3, 1)),
                                   'Decision_Tree__splitter': ['best', 'random'],
                                   'Decision_Tree__criterion' :['gini', 'entropy', 'log_loss']}, 
                                  area_under_precision_recall_curve_scorer,
                                  'area under precision recall curve',
                                  search_type = 'grid')

For model - Decision_Tree grid search:
----------------------------------------------------------
Best mean area under precision recall curve:
Train set: 0.893
Validation set: 0.886
----------------------------------------------------------
{'Decision_Tree__criterion': 'entropy', 'Decision_Tree__max_depth': 2, 'Decision_Tree__max_features': None, 'Decision_Tree__min_samples_leaf': 1, 'Decision_Tree__splitter': 'best'}


The performance improved by 0.7% and as expected PCA didn't help too much but yeah it did. The decision boundaries are slightly non-linear, already many columns went through PCA and thus, this was unhandalable by PCA. This problem should exist with ensemble models that use trees as well but maybe the voting part might help.  
From what we have seen till now is that the decision boundary for most part is linear but on the extreme ends it tends to become non-linear and thus, a model like SVM which can handle non-linearity and models parametric equations could work or something like a deep learning model also might work. For now, I will try with XGBoost as well, if things don't workout with that also then the problem in hand might not be handeled by tree models appropriately.

But first I will try decision tree without any column trasnformation for V1 to V28 columns. Since the model can adjust its complexity I  think it shouldn't make as such any difference in performance due to the transformation and I want to see that for myself.

##### 8. Decision tree without V1 to V28 transformations and PCA

In [27]:
# Performing random search cross-validation

find_best_cv_params(X_resampled,
                    Y_resampled,
                    Pipeline([('column_transformer', column_transformer_2), 
                              ('pca', PCA(random_state = 42)),
                              ('standard_scaler', standard_scaler),
                              ('Decision_Tree', DecisionTreeClassifier(class_weight = 'balanced', random_state = 123))]),                    
                    {'Decision_Tree__max_features': [None, 'sqrt', 'log2'],
                     'Decision_Tree__min_samples_leaf': list(range(5, 51, 5)),
                     'Decision_Tree__max_depth' : list(range(1, 3, 1)),
                     'Decision_Tree__splitter': ['best', 'random'],
                     'Decision_Tree__criterion' :['gini', 'entropy', 'log_loss']}, 
                     area_under_precision_recall_curve_scorer,
                     'area under precision recall curve',
                     n_iter = 100)

For model - Decision_Tree random search:
----------------------------------------------------------
Best mean area under precision recall curve:
Train set: 0.895
Validation set: 0.89
----------------------------------------------------------


{'Decision_Tree__splitter': 'best',
 'Decision_Tree__min_samples_leaf': 35,
 'Decision_Tree__max_features': None,
 'Decision_Tree__max_depth': 2,
 'Decision_Tree__criterion': 'entropy'}

In [28]:
# Performing grid search cross-validation

best_params = find_best_cv_params(X_resampled,
                                  Y_resampled,
                                  Pipeline([('column_transformer', column_transformer_2), 
                                          ('pca', PCA(random_state = 42)),
                                          ('standard_scaler', standard_scaler),
                                          ('Decision_Tree', DecisionTreeClassifier(class_weight = 'balanced', random_state = 123))]),                    
                                  {'Decision_Tree__max_features': [None, 'sqrt', 'log2'],
                                  'Decision_Tree__min_samples_leaf': list(range(25, 46, 2)),
                                  'Decision_Tree__max_depth' : list(range(1, 3, 1)),
                                  'Decision_Tree__splitter': ['best', 'random'],
                                  'Decision_Tree__criterion' :['gini', 'entropy', 'log_loss']}, 
                                  area_under_precision_recall_curve_scorer,
                                  'area under precision recall curve',
                                  search_type = 'grid')

For model - Decision_Tree grid search:
----------------------------------------------------------
Best mean area under precision recall curve:
Train set: 0.895
Validation set: 0.89
----------------------------------------------------------
{'Decision_Tree__criterion': 'entropy', 'Decision_Tree__max_depth': 2, 'Decision_Tree__max_features': None, 'Decision_Tree__min_samples_leaf': 29, 'Decision_Tree__splitter': 'best'}


The performance increased by 0.4% compared to when the trasformations were not their in place. I guess it doesn't always work for our better to have normally distributed predictor features. Definitely need to think on it more, if anyone knows what might have happened here do let me know in the comments or through my socials.

##### 9. XGBoost with class weight balancing, without V1 to V28 transformations and PCA

In [29]:
# Performing random search cross-validation

find_best_cv_params(X_resampled,
                    Y_resampled,
                    Pipeline([('column_transformer', column_transformer_2), 
                              ('pca', PCA(random_state = 42)),
                              ('standard_scaler', standard_scaler),
                              ('xgboost',XGBClassifier(objective='binary:logistic', silent=True, seed = 123, class_weight = 'balanced', error_score='raise'))]),                    
                    {'xgboost__gamma': [0.001, 0.01, 0.5, 1, 1.5, 2, 5, 10, 15],
                    'xgboost__learning_rate' : np.linspace(0.05, 1, 20), 
                    'xgboost__n_estimators' : list(range(2, 4)), 
                    'xgboost__subsample': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
                    'xgboost__colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
                    'xgboost__lambda': np.logspace(-4, 4, 10),
                    'xgboost__max_depth': [1, 2, 3,]},
                    area_under_precision_recall_curve_scorer,
                    'area under precision recall curve',
                    n_iter = 100)

For model - xgboost random search:
----------------------------------------------------------
Best mean area under precision recall curve:
Train set: 0.93
Validation set: 0.925
----------------------------------------------------------


{'xgboost__subsample': 0.9,
 'xgboost__n_estimators': 3,
 'xgboost__max_depth': 2,
 'xgboost__learning_rate': np.float64(0.3),
 'xgboost__lambda': np.float64(0.000774263682681127),
 'xgboost__gamma': 0.001,
 'xgboost__colsample_bytree': 0.6}

In [31]:
# Performing grid search cross-validation

best_params = find_best_cv_params(X_resampled,
                                  Y_resampled,
                                  Pipeline([('column_transformer', column_transformer_2), 
                                              ('pca', PCA(random_state = 42)),
                                              ('standard_scaler', standard_scaler),
                                              ('xgboost',XGBClassifier(objective='binary:logistic', silent=True, seed = 123, class_weight = 'balanced', error_score='raise'))]),                    
                                  {'xgboost__gamma': [0.0001, 0.001, 0.01],
                                  'xgboost__learning_rate' : [0.25, 0.3, 0.35], 
                                  'xgboost__n_estimators' : list(range(2, 5)), 
                                  'xgboost__subsample': [0.85, 0.9, 0.95],
                                  'xgboost__colsample_bytree': [0.55, 0.6, 0.65],
                                  'xgboost__lambda': [0.0006, 0.0008, 0.001],
                                  'xgboost__max_depth': [2, 3]},
                                  area_under_precision_recall_curve_scorer,
                                  'area under precision recall curve',
                                  search_type = 'grid')

For model - xgboost grid search:
----------------------------------------------------------
Best mean area under precision recall curve:
Train set: 0.925
Validation set: 0.925
----------------------------------------------------------
{'xgboost__colsample_bytree': 0.6, 'xgboost__gamma': 0.0001, 'xgboost__lambda': 0.0006, 'xgboost__learning_rate': 0.25, 'xgboost__max_depth': 2, 'xgboost__n_estimators': 3, 'xgboost__subsample': 0.85}


The performance is just slightly better than logistic regresison model as the perforamnce improved by 0.1% comparatively but the overfitting is definitely reduced to the point that both train and validation set have same performance.  
Last thing that I want to try will be what if we didn't drop any columns? This is ust to unerstand if the use of statistical tests has helped or not.

##### 10. XGBoost with class weight balancing, PCA, without V1 to V28 transformations and dropping of columns

In [32]:
# Performing random search cross-validation
column_transformer_3 = ColumnTransformer([('double_log_transform', double_log_transformer, double_log_transform_columns)],
                                          remainder = 'passthrough')

find_best_cv_params(X_resampled,
                    Y_resampled,
                    Pipeline([('column_transformer', column_transformer_3), 
                              ('pca', PCA(random_state = 42)),
                              ('standard_scaler', standard_scaler),
                              ('xgboost',XGBClassifier(objective='binary:logistic', silent=True, seed = 123, class_weight = 'balanced', error_score='raise'))]),                    
                    {'xgboost__gamma': [0.001, 0.01, 0.5, 1, 1.5, 2, 5, 10, 15],
                    'xgboost__learning_rate' : np.linspace(0.05, 1, 20), 
                    'xgboost__n_estimators' : list(range(2, 4)), 
                    'xgboost__subsample': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
                    'xgboost__colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
                    'xgboost__lambda': np.logspace(-4, 4, 10),
                    'xgboost__max_depth': [1, 2, 3,]},
                    area_under_precision_recall_curve_scorer,
                    'area under precision recall curve',
                    n_iter = 100)

For model - xgboost random search:
----------------------------------------------------------
Best mean area under precision recall curve:
Train set: 0.929
Validation set: 0.92
----------------------------------------------------------


{'xgboost__subsample': 0.7,
 'xgboost__n_estimators': 3,
 'xgboost__max_depth': 2,
 'xgboost__learning_rate': np.float64(0.3),
 'xgboost__lambda': np.float64(0.3593813663804626),
 'xgboost__gamma': 15,
 'xgboost__colsample_bytree': 0.9}

In [33]:
# Performing grid search cross-validation

best_params = find_best_cv_params(X_resampled,
                                  Y_resampled,
                                  Pipeline([('column_transformer', column_transformer_3), 
                                              ('pca', PCA(random_state = 42)),
                                              ('standard_scaler', standard_scaler),
                                              ('xgboost',XGBClassifier(objective='binary:logistic', silent=True, seed = 123, class_weight = 'balanced', error_score='raise'))]),                    
                                  {'xgboost__gamma': [10, 15, 20],
                                  'xgboost__learning_rate' : [0.25, 0.3, 0.35], 
                                  'xgboost__n_estimators' : list(range(2, 5)), 
                                  'xgboost__subsample': [0.65, 0.7, 0.75],
                                  'xgboost__colsample_bytree': [0.85, 0.9, 0.95],
                                  'xgboost__lambda': [0.26, 0.36, 0.46],
                                  'xgboost__max_depth': [2, 3]},
                                  area_under_precision_recall_curve_scorer,
                                  'area under precision recall curve',
                                  search_type = 'grid')

For model - xgboost grid search:
----------------------------------------------------------
Best mean area under precision recall curve:
Train set: 0.926
Validation set: 0.924
----------------------------------------------------------
{'xgboost__colsample_bytree': 0.9, 'xgboost__gamma': 20, 'xgboost__lambda': 0.26, 'xgboost__learning_rate': 0.35, 'xgboost__max_depth': 3, 'xgboost__n_estimators': 3, 'xgboost__subsample': 0.65}


The performance has slightly decreased and this is a good indication that our statistical tests worked fine and including them has decreased the performance slightly as the algorithm might be capturing the noise within them.

##### Conclusion of experimentation

**In conlusion of the experimentation:**  
- Normalizaing transformations or skewness reducing transformations are not effective here at least.  
- Our feature selection stood up to its reputation and thus, proved to be benfecial.  
- Simpler models like logistic regression are performing almost same as much more complex models like xgboost while decision tree performed relatively quite poorly.  
- PCA helped improve performance of tree models.

**Going on ahead:**  
- Will use the conclusions from above to start training our final models which can then ultimately be evaluate don our unseen test set.  
- I will try logistic regression, XGBoost, Support vector machine and deep learning model.

#### Final training

##### 1. Logistic Regression

In [34]:
column_transformer_final = ColumnTransformer([('drop', 'drop', drop_columns)],
                                              remainder = 'passthrough')

# Performing random search cross-validation

find_best_cv_params(X_resampled,
                    Y_resampled,
                    Pipeline([('column_transformer', column_transformer_final), 
                              ('standard_scaler', standard_scaler),
                              ('Logistic_Regression', LogisticRegression(class_weight = 'balanced', n_jobs = -1, random_state = 123))]),
                    {'Logistic_Regression__penalty' : ['l1', 'l2', 'elasticnet', None],
                     'Logistic_Regression__C' : np.logspace(-4, 4, 20), 
                     'Logistic_Regression__solver': ['lbfgs', 'newton-cg', 'sag', 'saga'],
                     'Logistic_Regression__l1_ratio': np.linspace(0.1, 1, 9, endpoint = False)}, 
                    area_under_precision_recall_curve_scorer,
                    'area under precision recall curve',
                    n_iter = 50)

For model - Logistic_Regression random search:
----------------------------------------------------------
Best mean area under precision recall curve:
Train set: 0.936
Validation set: 0.924
----------------------------------------------------------


{'Logistic_Regression__solver': 'lbfgs',
 'Logistic_Regression__penalty': None,
 'Logistic_Regression__l1_ratio': np.float64(0.6),
 'Logistic_Regression__C': np.float64(206.913808111479)}

While doing final trainings, I will try to store the model and best grid search parameters so that I can use them later on for final training on the full train set and also, for my end-to-end deployment.

In [35]:
models = []

In [37]:
models.append({'name': 'Logistic_Regression',
               'pipeline': Pipeline([('column_transformer', column_transformer_final), 
                                     ('standard_scaler', standard_scaler),
                                     ('Logistic_Regression', LogisticRegression(class_weight = 'balanced', n_jobs = -1, random_state = 123))])})

models[0]['best_params'] = find_best_cv_params(X_resampled,
                                               Y_resampled,
                                               models[0]['pipeline'],
                                               {'Logistic_Regression__penalty' : [None],
                                                'Logistic_Regression__solver': ['lbfgs']}, 
                                               area_under_precision_recall_curve_scorer,
                                               'area under precision recall curve',
                                               search_type = 'grid')

For model - Logistic_Regression grid search:
----------------------------------------------------------
Best mean area under precision recall curve:
Train set: 0.936
Validation set: 0.924
----------------------------------------------------------
{'Logistic_Regression__penalty': None, 'Logistic_Regression__solver': 'lbfgs'}


##### 2. Support Vector machine

In [40]:
# Performing random search cross-validation

find_best_cv_params(X_resampled,
                    Y_resampled,
                    Pipeline([('column_transformer', column_transformer_final), 
                              ('standard_scaler', standard_scaler),
                              ('Support_Vecotor_Classifier', SVC(probability = True, class_weight = 'balanced', random_state = 42))]),
                    {'Support_Vecotor_Classifier__C' : np.logspace(-4, 4, 20), 
                     'Support_Vecotor_Classifier__gamma' : np.logspace(-4, 0, 20), 
                     'Support_Vecotor_Classifier__kernel': ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'],
                     'Support_Vecotor_Classifier__degree': [1, 2, 3, 4]}, 
                    area_under_precision_recall_curve_scorer,
                    'area under precision recall curve',
                    n_iter = 50)

KeyboardInterrupt: 

### References

- https://www.kdnuggets.com/2023/01/7-smote-variations-oversampling.html
- https://www.analyticsvidhya.com/blog/2020/10/overcoming-class-imbalance-using-smote-techniques/#h-dealing-with-imbalanced-data
- https://scikit-learn.org/stable/
- https://www.youtube.com/@krishnaik06  
- Hands on Machine Learning with Scikit-Learn & TensorFlow by Aurélien Géron (O'Reilly). CopyRight 2017 Aurélien Géron